In [1]:
import os
import sys

_PATH = '/Users/winston/mygitrep/sys23/'  # TODO
os.chdir(_PATH)
sys.path.append(_PATH)

In [2]:
import warnings
warnings.simplefilter("ignore")
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
# seaborn.set_style("darkgrid")
seaborn.set_style("whitegrid")
plt.rc("figure", figsize=(16, 6))
# plt.rc("figure", figsize=(6, 3))
plt.rc("savefig", dpi=90)
# plt.rc("font", family="sans-serif")
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rc("font", size=12)
# plt.rc("font", size=10)
plt.rcParams["date.autoformatter.hour"] = "%H:%M:%S"

In [3]:
from api.load_tushare import conf_init, next_calendar_date, TushareLoader

import pandas as pd
import numpy as np

config_path = f'{_PATH}/config_stk.yaml'
conf = conf_init(conf_path=config_path)
print(conf.keys())

begin_date = '2015-01-01'
# begin_date = '2018-01-01'
end_date = '2022-12-31'
# end_date = '2023-03-31'

tgt_path = conf['path']['factors_csv']
os.makedirs(tgt_path, exist_ok=True)

dict_keys(['status', 'tushare_token', 'api_path', 'path', 'year_length', 'csi_pool', 'update_cache_demo'])


In [4]:
from tqdm import tqdm

def cal_ic_stat(data):
    """获取IC的统计指标"""
    from scipy import stats
    
    data = data.dropna()
    t_value, p_value = stats.ttest_1samp(data, 0)  # 计算ic的t
    pdata = data[data >= 0]
    ndata = data[data < 0]
    data_stat = list(zip(
        data.mean(), data.std(), data.skew(), data.kurt(), t_value, p_value,
        pdata.mean(), ndata.mean(), pdata.std(), ndata.std(),
        ndata.isna().mean(), pdata.isna().mean(), data.mean() / data.std()
    ))
    data_stat = pd.DataFrame(data_stat).T
    data_stat.columns = data.columns
    data_stat.index = ['mean', 'std', 'skew', 'kurt', 't_value', 'p_value',
                       'mean+', 'mean-', 'std+', 'std-', 'pos ratio', 'neg ratio', 'IR']
    #
    return data_stat

In [11]:
src_path = conf['path']['factors_res']
folders = [_ for _ in os.listdir(src_path) if 'COMB_' in _]
stat_ic_rank = []
stat_ic = []
for folder in tqdm(folders):
    path_fmt = src_path + folder + '/{}'
    fname = folder.replace('[NEW]', '').replace('COMB_', '')
    df = pd.read_csv(path_fmt.format('截面RankIC.csv'), index_col=0)
    stat_ic_rank.append(cal_ic_stat(df).T.stack().rename(fname))
    df = pd.read_csv(path_fmt.format('截面IC.csv'), index_col=0)
    stat_ic.append(cal_ic_stat(df).T.stack().rename(fname))
stat_ic = pd.concat(stat_ic, axis=1)
stat_ic.to_csv(conf['path']['factors_res'] + f'COMB_reports_IC[{len(folders)}].csv')
stat_ic_rank = pd.concat(stat_ic_rank, axis=1)
stat_ic_rank.to_csv(conf['path']['factors_res'] + f'COMB_reports_RankIC[{len(folders)}].csv')

100%|█████████████████████████████████████████████| 6/6 [00:00<00:00, 64.73it/s]


In [7]:
src_path = conf['path']['factors_res']
folders = [_ for _ in os.listdir(src_path) if '_residsectorbarrar[NEW]' in _]
stat_ic_rank = []
stat_ic = []
for folder in tqdm(folders):
    path_fmt = src_path + folder + '/{}'
    fname = folder.replace('[NEW]', '')
    df = pd.read_csv(path_fmt.format('截面RankIC.csv'), index_col=0)
    stat_ic_rank.append(cal_ic_stat(df).T.stack().rename(fname))
    df = pd.read_csv(path_fmt.format('截面IC.csv'), index_col=0)
    stat_ic.append(cal_ic_stat(df).T.stack().rename(fname))
stat_ic = pd.concat(stat_ic, axis=1)
stat_ic.to_csv(conf['path']['factors_res'] + f'reports_IC[{len(folders)}].csv')
stat_ic_rank = pd.concat(stat_ic_rank, axis=1)
stat_ic_rank.to_csv(conf['path']['factors_res'] + f'reports_RankIC[{len(folders)}].csv')

100%|███████████████████████████████████████████| 32/32 [00:00<00:00, 86.09it/s]


In [10]:
stat_ic_rank.round(3).to_clipboard()